In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [7]:
endpoint = "http://dbpedia.org/sparql"

query = """
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT DISTINCT ?person ?birthdate ?birthyear ?activeyear ?activeyears ?deathyear ?deathage
WHERE { 
    ?person rdf:type dbo:Philosopher.

    OPTIONAL { ?person dbo:birthDate ?birthdate }
    OPTIONAL { ?person dbo:birthYear ?birthyear }
    OPTIONAL { ?person dbo:activeYear ?activeyear }
    OPTIONAL { ?person dbo:activeYears ?activeyears }
    OPTIONAL { ?person dbo:deathYear ?deathyear }
    OPTIONAL { ?person dbo:deathAge ?deathage }

}
ORDER BY ?birthdate
"""

sparql = SPARQLWrapper(endpoint)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

print len(results["results"]["bindings"])

1920


In [37]:
import json

with open('dates.birthdeath.dbpedia.json', 'w') as outfile:
    json.dump(results["results"]["bindings"], outfile, sort_keys=True, indent=4, separators=(',', ': '))

In [9]:
import pandas as pd

In [77]:
names = [ p["person"]["value"].replace("http://dbpedia.org/resource/", "") for p in results["results"]["bindings"]]

df = pd.DataFrame(columns=["name", "birthyear", "birthdate", "deathyear"])

df["name"] = names

df = df.set_index("name")

for p in results["results"]["bindings"]:
  k = [ k for k in p.keys() if k != "person" ]
  n = p["person"]["value"].replace("http://dbpedia.org/resource/", "")
    
  df.set_value(n, k, [ p[x]["value"] for x in p if x in k])

df.head(10)

,birthyear,birthdate,deathyear
name,,,
Syed_Abdul_Hakim,1874,NaN,1944
Moses_ibn_Ezra,1055,NaN,1138
Francisco_Sanches,1550,NaN,1623
Sibawayh,0760,NaN,0796
Solomon_ibn_Gabirol,1021,NaN,1058
Abu_Yusuf,0113,NaN,0798
Sufyan_al-Thawri,0716,NaN,0161
George_Ripley_(alchemist),1415,NaN,1490
Euthymius_of_Athos,0955,NaN,1024


In [79]:
df.to_csv("dates.birthdeath.csv", encoding="utf-8")